In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Zach Flowers, Murtaza Safdari <musafdar@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

# Imports

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
import logging
import i2c_gui
import i2c_gui.chips
from i2c_gui.usb_iss_helper import USB_ISS_Helper
from i2c_gui.fpga_eth_helper import FPGA_ETH_Helper
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
# import time
from tqdm import tqdm
# from i2c_gui.chips.etroc2_chip import register_decoding
import os, sys
import multiprocessing
os.chdir(f'/home/{os.getlogin()}/ETROC2/ETROC_DAQ')
import run_script
import importlib
importlib.reload(run_script)
import datetime
import pandas
from pathlib import Path
import subprocess
import sqlite3
from fnmatch import fnmatch
import scipy.stats as stats
import hist
import mplhep as hep
plt.style.use(hep.style.CMS)
from notebooks.notebook_helpers import *

from scripts.log_action import log_action_v2
from time import sleep

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure()
plt.show()

# Set defaults

In [ ]:
# !!!!!!!!!!!!
# It is very important to correctly set the chip name, this value is stored with the data
chip_names = ["ET2p03_BARE2"]
# chip_names = ["ET2p01_BARE10"]
chip_fignames = chip_names
chip_figtitles = chip_names

# 'The port name the USB-ISS module is connected to. Default: /dev/ttyACM0'
port = "/dev/ttyACM5"
# I2C addresses for the pixel block and WS
chip_addresses = [0x60] 
ws_addresses = [None]

hostname = "192.168.2.3"

today = datetime.date.today().isoformat()

polarity = "0x4023" # ?
# polarity = "0x000f"

LED_page = "0x0000"
active_channel = "0x0001"
delay = "0x1800"

fig_outdir = Path('../ETROC-figures')
fig_outdir = fig_outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
fig_outdir.mkdir(exist_ok=True)
fig_path = str(fig_outdir)

## Make i2c_connection class object

In [ ]:
# i2c_conn = self, port, chip_addresses, chip_names, chip_fc_delays
i2c_conn = i2c_connection(port,chip_addresses,ws_addresses,chip_names,[("1","1")])

## Disable Pixels

pixels will be enabled when needed by later function calls

In [ ]:
#log_action_v2(Path("../i2c_gui"), "Config", "", "Disabling pixels")
i2c_conn.config_chips('00100000')
# i2c_conn.config_chips('00100111')
i2c_conn.config_chips('00001000')
# i2c_conn.config_chips('00000100')

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure()
plt.show()

In [ ]:
# histdir = Path('../ETROC-History/')
# histdir = Path('/media/daq/X9/DESY_Jun2024_AIDA/ETROC-History/')
histdir = Path('/home/daq/ETROC2/ETROC-History/CERN_Aug_2024_CC_Spirale3')
histdir.mkdir(exist_ok=True)
histfile = histdir / 'BaselineHistory.sqlite'
figure_title = "-24.5C, 0 MRad, A: 1.32V, D: 1.2V"
i2c_conn.save_baselines(chip_fignames, fig_title=figure_title, fig_path=fig_path, histdir=histdir, histfile=histfile, save_notes=figure_title)

## Define pixels of interest

# Noise Data Taking

In [ ]:
# row_list = [0, 0, 3, 3, 12, 12]
# col_list = [14, 2, 14, 2, 14, 2]

row_list = [0, 0, 3, 3, 12, 12]
col_list = [14, 0, 14, 0, 14, 0]

# row_list = [0, 0, 3, 12, 12]
# col_list = [14, 0, 14, 14, 0]

# row_list = [12, 12]
# col_list = [14, 0]

scan_list = list(zip(row_list, col_list))
print(scan_list)

## Finding the Turn-On Point of the Pixel

In [ ]:
#log_action_v2(Path("../i2c_gui"), "Config", "", "Finding pixel turn-on point")

# pixel_turnon_points(i2c_conn, chip_addresses[0], chip_fignames[0], LED_page, delay, active_channel, polarity, scan_list, verbose=False, calibrate=True, today=today, hostname=hostname)

## Noise Peak Scans

In [ ]:
#log_action_v2(Path("../i2c_gui"), "Config", "", "Noise peak scan")
polarity = "0x0023"
trigger_bit_noisescan(i2c_conn, chip_addresses[0], chip_fignames[0], LED_page, delay, active_channel, polarity, scan_list, verbose=False, pedestal_scan_step=1, attempt='_quietCB', busyCB=False, today=today, hostname=hostname)

## Noise Peak Scan Plot

In [ ]:
trigger_bit_noisescan_plot(i2c_conn, chip_addresses[0], chip_figtitles[0], chip_fignames[0], scan_list, attempt='_quietCB', tag=" Quiet CB", autoBL=True, gaus=True, today=today)

## Finding the Peak DAC Values per Charge

In [ ]:
# QInjEns = [5, 6, 8, 15, 22, 27]
QInjEns = [6, 8, 10, 15, 22]
# QInjEns = [15]

In [ ]:
#log_action_v2(Path("../i2c_gui"), "Config", "", "Finding pixel turn-off point")
polarity = "0x0023"
pixel_turnoff_points(i2c_conn, chip_addresses[0], chip_fignames[0], LED_page, delay, active_channel, polarity, scan_list, verbose=False, QInjEns=QInjEns, today=today, hostname=hostname, triggerbit=False, attempt="_data")

## Q vs Peak DAC Plots

In [ ]:
charge_peakDAC_plot(i2c_conn, chip_addresses[0], chip_figtitles[0], chip_fignames[0], scan_list, QInjEns, today=today, attempt="_data")

## Q v DAC Full Scan

### This takes a long time as it takes the data for the "money plots"

In [ ]:
#log_action_v2(Path("../i2c_gui"), "Config", "", "Full Q vs DAC Scan")

# full_scurve_scan(i2c_conn, chip_addresses[0], chip_figtitles[0], chip_fignames[0], s_flag=LED_page, d_flag=delay, a_flag=active_channel, p_flag=polarity, scan_list=scan_list, verbose=False, QInjEns=QInjEns, pedestal_scan_step=1, attempt='', tp_tag='', today=today, allon=False, neighbors=False, hostname=hostname)
polarity = "0x4023"
full_scurve_scan(i2c_conn, chip_addresses[0], chip_fignames[0], s_flag=LED_page, d_flag=delay, a_flag=active_channel, p_flag=polarity, scan_list=scan_list, verbose=False, QInjEns=QInjEns, pedestal_scan_step=1, attempt='', tp_tag='_data', today=today, allon=False, neighbors=False, hostname=hostname, run_options="--compressed_binary --skip_translation",timePerPixel=2, deadTime=1, BL_offset=-2)

# full_scurve_scan(i2c_conn, chip_addresses[0], chip_fignames[0], s_flag=LED_page, d_flag=delay, a_flag=active_channel, p_flag=polarity, scan_list=scan_list, verbose=False, QInjEns=QInjEns, pedestal_scan_step=40, attempt='', tp_tag='_data', today=today, allon=False, neighbors=False, hostname=hostname, run_options="--suppress_fillers --skip_binary --clear_fifo",timePerPixel=4, deadTime=1, BL_offset=-2)

## Plotting Full S-Curves

In [ ]:
process_scurves(chip_figtitles[0], chip_fignames[0], QInjEns, scan_list, today=today)

# Calibrate FC for all I2C

In [ ]:
for chip_address in chip_addresses[:]:
    i2c_conn.asyResetGlobalReadout(chip_address, chip=None)
    i2c_conn.asyAlignFastcommand(chip_address, chip=None)

# Calibrate PLL

In [ ]:
for chip_address in chip_addresses[:]:
    i2c_conn.calibratePLL(chip_address, chip=None)

# Disconnect I2C Device

In [ ]:
del i2c_conn